# Changing the World Type of One Dimension

I have been having really bad luck finding Netherite. There are two possibilities:

1. I have unrealistic expectations, given that before this world, I _literally cheated_ by creating a copy of my world and putting it into Creative mode, writing down all the Ancient Debris coordinates, and then mining straight to the best caches.
2. [The BCLib mod](https://github.com/quiqueck/BCLib/) is messing with ore generation

Even though I'm pretty sure I know which of the two theories is more likely, I'm still going to explore disabling the "BetterX" world generation settings for The Nether and _just_ the Nether, as, well, I don't actually _want_ any mods changing The Nether.

[BetterEnd](https://github.com/quiqueck/BetterEnd) is easily my second favorite mod, after only [Iris](https://irisshaders.net/), but while I appreciate thee creativitiy they put into [the BetterNether mod](https://github.com/quiqueck/BetterNether), I don't generally use it, because IMO the Nether is better enough.

But BCLib's default settings cause any world--old or new--to use the "BetterX" world type for both non-Overworld dimensions, and that's caused discrepancies when scouting out my world using tools like [chunkbase](https://www.chunkbase.com/), and the bottom line is, _I really don't want BCLib altering the Nether_, at least not any more than it already has.

So the goal of this notebook is to open up`level.dat` and find out explicitly to what degree I can make it so that I continue using "BetterX" in the End but not in the Nether.

## Changing Mod Settings

The first thing to do is to go into the BCLib config and make sure that any settings I change here don't get reverted when I next open my world.

Following my nose, I'm guessing the keys to change are:

#### `server.json`
- `auto_sync`
  - `forceBetterXPreset`: `false`

#### `generator.json`
- `options`
  - `biomeSource`
    - `fixNetherBiomeSource`: `false`

## Imports, Setup and Macros

In [1]:
import json
from os import environ
from pathlib import Path
from typing import Any, Collection, Dict, Set

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
def format_file_size(path: Path) -> str:
    """Print the size of the specified file in
    human-readible form (KB / MB / GB)

    Parameters
    ----------
    path : Path
        The path to the file

    Returns
    -------
    str
        A prettily formatted file size

    Notes
    -----
    I would be shocked if there isn't a utility already built
    into the standard library to do this, but all I could find
    via Googling was a bunch of recipes and examples
    """
    size: float = path.stat().st_size  # in bytes
    for unit in ("B", "KB", "MB", "GB"):
        if size < 1024 / 2:
            return f"{size:.1f} {unit}"
        size = size / 1024
    return f"{size} TB"

In [3]:
def summarize_keystore(keystore: Dict[str, Any]) -> None:
    """Display a summary of the contents of a key-value store

    Parameters
    ----------
    keystore : dict
        The keystore to summarize

    Returns
    -------
    None
    """

    def _summarize_keystore(keystore: Dict[str, Any]) -> str:
        summary = ""
        for k, v in keystore.items():
            summary += f"\n - `{k}` : "
            if isinstance(v, (str, nbt.TAG_String)):
                summary += f'`"{v}"`'
            elif not isinstance(v, Collection):
                summary += f"`{str(v)}`"
            else:
                length = len(v)
                if 0 < length < 3:
                    summary += "\n"
                    if not isinstance(v, Dict):
                        v = {i: item for i, item in enumerate(v)}
                    summary += "\n".join(
                        (f"\t{line}" for line in _summarize_keystore(v).split("\n"))
                    )
                else:
                    summary += f"({len(v)} items)"
        return summary

    display(Markdown(_summarize_keystore(keystore)))

In [4]:
save_folder = Path(environ["SAVE_PATH"])

# make sure this is set correctly
for path in sorted(save_folder.glob("*")):
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")

- DIM-1 (folder)
- DIM1 (folder)
- advancements (folder)
- data (folder)
- datapacks (folder)
- entities (folder)
- icon.png (8.6 KB)
- level.dat (20.0 KB)
- level.dat_old (20.0 KB)
- level11326231134829879582.dat (0.0 B)
- level14927678268100607923.dat (0.0 B)
- level1786655981796876926.dat (0.0 B)
- level4463453738642305340.dat (0.0 B)
- level6425531070021529407.dat (0.0 B)
- level7443636089696258371.dat (0.0 B)
- level8832581565660323154.dat (0.0 B)
- playerdata (folder)
- poi (folder)
- region (folder)
- serverconfig (folder)
- session.lock (3.0 B)
- stats (folder)
- worldgen_settings_export.json (1.5 KB)


## Load up the ole `level.dat`

In [5]:
%%time
level = nbt.NBTFile(save_folder / "level.dat")

CPU times: user 25.3 ms, sys: 0 ns, total: 25.3 ms
Wall time: 24.6 ms


In [6]:
summarize_keystore(level["Data"])


 - `Difficulty` : `3`
 - `thunderTime` : `136444`
 - `BorderSize` : `59999968.0`
 - `LastPlayed` : `1667356546044`
 - `allowCommands` : `0`
 - `BorderCenterX` : `0.0`
 - `initialized` : `1`
 - `BorderWarningBlocks` : `5.0`
 - `hardcore` : `0`
 - `version` : `19133`
 - `ServerBrands` : 
	
	 - `0` : `"forge"`
	 - `1` : `"fabric"`
 - `SpawnX` : `-592`
 - `GameType` : `0`
 - `BorderSafeZone` : `5.0`
 - `SpawnAngle` : `0.0`
 - `LevelName` : `"Esha Ness"`
 - `Time` : `63109557`
 - `ScheduledEvents` : (19 items)
 - `clearWeatherTime` : `0`
 - `WanderingTraderId` : (4 items)
 - `BorderDamagePerBlock` : `0.2`
 - `WanderingTraderSpawnDelay` : `7200`
 - `thundering` : `0`
 - `WasModded` : `1`
 - `BorderWarningTime` : `15.0`
 - `WanderingTraderSpawnChance` : `50`
 - `SpawnY` : `68`
 - `SpawnZ` : `144`
 - `BorderSizeLerpTime` : `0`
 - `raining` : `0`
 - `WorldGenSettings` : (4 items)
 - `rainTime` : `39986`
 - `DataPacks` : 
	
	 - `0` : `"Disabled"`
	 - `1` : `"Enabled"`
 - `DataVersion` : `3120`
 - `GameRules` : (36 items)
 - `DragonFight` : (6 items)
 - `Player` : (46 items)
 - `DifficultyLocked` : `0`
 - `DayTime` : `87432669`
 - `BorderCenterZ` : `0.0`
 - `BorderSizeLerpTarget` : `59999968.0`
 - `Version` : (4 items)
 - `CustomBossEvents` : 
	
	 - `0` : `"multiplayer_sleep:preview"`
	 - `1` : `"multiplayer_sleep:progress"`

So indeed, I see a key for "WorldGenSettings"

In [7]:
summarize_keystore(level["Data"]["WorldGenSettings"])


 - `bonus_chest` : `1`
 - `seed` : `5569739623460313880`
 - `generate_features` : `1`
 - `dimensions` : (3 items)

In [8]:
summarize_keystore(level["Data"]["WorldGenSettings"]["dimensions"])


 - `minecraft:the_nether` : 
	
	 - `0` : `"type"`
	 - `1` : `"generator"`
 - `minecraft:overworld` : 
	
	 - `0` : `"type"`
	 - `1` : `"generator"`
 - `minecraft:the_end` : 
	
	 - `0` : `"type"`
	 - `1` : `"generator"`

In [9]:
for dimension, settings in level["Data"]["WorldGenSettings"]["dimensions"].items():
    display(Markdown(f"#### {dimension}"))
    display(Markdown(f"Type: {str(settings['type'])}"))
    summarize_keystore(settings["generator"])

#### minecraft:the_nether

Type: minecraft:the_nether


 - `type` : `"bclib:betterx"`
 - `biome_source` : (3 items)
 - `settings` : `"minecraft:nether"`

#### minecraft:overworld

Type: minecraft:overworld


 - `type` : `"minecraft:noise"`
 - `biome_source` : 
	
	 - `0` : `"type"`
	 - `1` : `"preset"`
 - `settings` : `"minecraft:overworld"`

#### minecraft:the_end

Type: minecraft:the_end


 - `type` : `"minecraft:noise"`
 - `biome_source` : (3 items)
 - `settings` : `"minecraft:end"`

Huh.

In [10]:
for dimension in ("nether", "end"):
    display(Markdown(f"#### {dimension}"))
    settings = level["Data"]["WorldGenSettings"]["dimensions"][
        f"minecraft:the_{dimension}"
    ]
    summarize_keystore(settings["generator"]["biome_source"])

#### nether


 - `type` : `"bclib:nether_biome_source"`
 - `seed` : `5569739623460313880`
 - `config` : (4 items)

#### end


 - `type` : `"bclib:end_biome_source"`
 - `seed` : `5569739623460313880`
 - `config` : (8 items)

_We need to go deeper!_

In [11]:
for dimension in ("nether", "end"):
    display(Markdown(f"#### {dimension}"))
    settings = level["Data"]["WorldGenSettings"]["dimensions"][
        f"minecraft:the_{dimension}"
    ]
    summarize_keystore(settings["generator"]["biome_source"]["config"])

#### nether


 - `biome_size_vertical` : `86`
 - `biome_size` : `256`
 - `use_vertical_biomes` : `1`
 - `map_type` : `"hex"`

#### end


 - `with_void_biomes` : `1`
 - `generator_version` : `"paulevs"`
 - `land_biomes_size` : `256`
 - `barrens_biomes_size` : `256`
 - `inner_void_radius_squared` : `1048576`
 - `void_biomes_size` : `256`
 - `map_type` : `"hex"`
 - `center_biomes_size` : `256`

Okay, so this confirms that we are using a nonstandard generator for the Nether, and also that the generators can be set separately for each dimension. So _theoretically_ I should be able to overwrite the worldgen settings and be on my way for any new chunks (or any existing chunks I prune with MCASelector).

To do this I'm gonna need a copy of a "good" config. Luckily, I have a vanilla world I can work with.

## Well, not quite "vanilla"

You see, I was experimenting with my own custom world gen settings the other day to see if I could have a single save with multiple overworlds and nethers, thus allowing me to create a kind of "parallel universe" map.

Results were mixed--you can _absolutely_ specify "extra" overworlds, nethers (and ends) using datapacks alone, and you can import the chunk data from another world save as these extra dimensions, but:
- you won't be able to light portals in the extra dimensions
- if you have an _already lit_ portal in those extra dimensions, they'll take you to the _original_ Nether / Overworld / End (that is, portal destination dimension is hard-coded).

So that's an idea I'll come back to. But in the meantime, because I wanted to minimize the number of variables, this was done in an almost purely vanilla instance--I'm not even using Sodium. So that should be the perfect "donor" for the NBT data I need.

<div class="alert alert-info" >
<h3>Quick note</h3>
Yes, I'm aware I can <i>export</i> worldgen settings from another world. What I'm not clear on is whether I can then <i>import them back</i> into an existing world in a permanent way.
</div>

In [12]:
donor_save_folder = Path(environ["SAVE_PATH"]) / ".." / "EverywhereAtOnce"

# make sure this is set correctly
for path in sorted(donor_save_folder.glob("*")):
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")

- DIM-1 (folder)
- DIM1 (folder)
- advancements (folder)
- data (folder)
- datapacks (folder)
- dimensions (folder)
- entities (folder)
- icon.png (10.1 KB)
- level.dat (2.3 KB)
- level.dat_old (2.3 KB)
- playerdata (folder)
- poi (folder)
- region (folder)
- session.lock (3.0 B)
- stats (folder)


In [13]:
%%time
donor_level = nbt.NBTFile(donor_save_folder / "level.dat")

CPU times: user 3.26 ms, sys: 0 ns, total: 3.26 ms
Wall time: 2.74 ms


In [14]:
display(Markdown("#### Nether Worldgen Settings"))
summarize_keystore(
    settings := donor_level["Data"]["WorldGenSettings"]["dimensions"][
        "minecraft:the_nether"
    ]
)
display(Markdown("#### Nether Generator Settings"))
summarize_keystore(settings := settings["generator"])
display(Markdown("#### Nether Biome Source Settings"))
summarize_keystore(settings := settings["biome_source"])

#### Nether Worldgen Settings


 - `generator` : (3 items)
 - `type` : `"minecraft:the_nether"`

#### Nether Generator Settings


 - `settings` : `"minecraft:nether"`
 - `biome_source` : 
	
	 - `0` : `"preset"`
	 - `1` : `"type"`
 - `type` : `"minecraft:noise"`

#### Nether Biome Source Settings


 - `preset` : `"minecraft:nether"`
 - `type` : `"minecraft:multi_noise"`

And now for the transplant...

In [15]:
level["Data"]["WorldGenSettings"]["dimensions"]["minecraft:the_nether"] = donor_level[
    "Data"
]["WorldGenSettings"]["dimensions"]["minecraft:the_nether"]

In [16]:
level.write_file(save_folder / "level.dat")

Let's see if that took.

In [18]:
%%time
level = nbt.NBTFile(save_folder / "level.dat")
summarize_keystore(
    level["Data"]["WorldGenSettings"]["dimensions"]["minecraft:the_nether"]
)


 - `generator` : (3 items)
 - `type` : `"minecraft:the_nether"`

CPU times: user 23.3 ms, sys: 0 ns, total: 23.3 ms
Wall time: 22.7 ms


In [18]:
summarize_keystore(
    level["Data"]["WorldGenSettings"]["dimensions"]["minecraft:the_nether"]["generator"]
)


 - `settings` : `"minecraft:nether"`
 - `biome_source` : 
	
	 - `0` : `"preset"`
	 - `1` : `"type"`
 - `type` : `"minecraft:noise"`

Yep. Looks good. Now let's see if BCLib will behave itself...

Nope. From the logs:

```
[22:29:54] [Render thread/INFO]: [bclib] Loading from builtin Registry
[22:29:54] [Render thread/INFO]: [bclib] Running Pre Patchers on /home/openbagtwo/.minecraft/saves/Esha Ness/.
[22:29:54] [Render thread/INFO]: [DataFixerAPI] Everything up to date
[22:29:55] [Render thread/INFO]: [fabric-registry-sync] Loaded registry data [file 1/4]
[22:29:55] [Render thread/INFO]: [bclib] Loading from datapack Registry
[22:29:55] [Render thread/INFO]: [bclib] Enforcing Correct Generator for minecraft:the_nether.
```

And indeed:

In [19]:
%%time
level = nbt.NBTFile(save_folder / "level.dat")
summarize_keystore(
    level["Data"]["WorldGenSettings"]["dimensions"]["minecraft:the_nether"]
)
summarize_keystore(
    level["Data"]["WorldGenSettings"]["dimensions"]["minecraft:the_nether"]["generator"]
)


 - `type` : `"minecraft:the_nether"`
 - `generator` : (3 items)


 - `type` : `"bclib:betterx"`
 - `biome_source` : (3 items)
 - `settings` : `"minecraft:nether"`

CPU times: user 49.2 ms, sys: 0 ns, total: 49.2 ms
Wall time: 47.8 ms


Bug or an additional setting I need to flip? Luckily I've got logger messages to help me find out.